# Title
[]()

In [2]:

import sys
sys.path.append('../src')
sys.path.append(r"/home/silvhua/custom_python")
from silvhua import *

In [11]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 200)
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Iteration 1

In [2]:
filename = 'pubmed_results_2024-04-06_235718.json'
filepath = '/home/silvhua/repositories/pubmed-search/data/'
dictionary_list = load_json(filename, filepath)

In [12]:
df = pd.DataFrame(dictionary_list)
df[['article_title', 'keywords']]

,article_title,keywords
0,Exploring home rehabilitation therapists' experiences of supporting older persons to physical exercise after acute hospitalization: a qualitative interview study.,"[Exercise, Older adults, Primary care, Qualitative study, Rehabilitation therapy, Transitional care]"
1,Current awareness and use of transthoracic echocardiography in evaluation of valvular heart disease in Kumamoto Prefecture -a report from the Kumamoto cardiovascular echocardiography standardizati...,"[Aortic stenosis, Transthoracic echocardiography, Valvular heart disease]"
2,Clinical and ultrasonographic evaluation of uninjured dominant shoulder in amateur rugby players vs a control group: a pilot study.,"[Clinical exam of the shoulder, Rotator cuff injuries, Rugby, Shoulder injuries, Tendon calcification, Ultrasonography]"
3,"A Letter to the Editor on ""Effect of Visual Biofeedback Obtained Using the Iowa Oral Performance Instrument on the Suprahyoid Muscle Activation Level During Effortful Swallowing Maneuver"".","[Attention, Effortful swallow, Electromyography]"
4,Effects of sporadic inclusion body myositis on skeletal muscle fibre type specific morphology and markers of regeneration and inflammation.,"[Immunology, Macrophages, Myonuclei, Myopathies, Satellite cells]"
5,High-flow nasal cannula during pulmonary rehabilitation for people with chronic obstructive pulmonary disease: A&#xa0;systematic review and meta-analysis.,"[chronic obstructive pulmonary disease, exercise therapy, high flow nasal cannula, oxygen inhalation therapy, quality of life, rehabilitation, systematic review]"
6,Non-Pharmacological Interventions for Treating Psoriatic Arthritis.,None
7,An integrative review of parent education approaches in sport: Considerations for program planning and evaluation.,"[families, intervention, parent involvement, re&#x2010;aim, youth sport]"
8,"The effects of telerehabilitation in adults with complex biventricular congenital heart conditions: protocol for a multi-centre, randomised controlled trial-CH-FIT.","[Aerobic fitness, Combined exercise, Complex congenital heart anomalies, Neurocognitive function, Telehealth]"
9,Exercise transiently increases the density of incipient blood clots in antiplatelet-treated lacunar stroke patients.,"[Clot microstructure, Coagulation, Exercise, Lacunar stroke patients, Thrombogenicity]"


In [16]:
# check the type of data contained in the 'article_title' column
df['article_title'].apply(lambda x: isinstance(x, list)).sum()

# df['keywords'] = df['keywords'].str.join(', ')
# df[['article_title', 'keywords']]

0

# add new files

In [3]:
from indexing_pipeline import *

/home/silvhua/.pubmed/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
class Index_Docs():

    def __init__(
            self, json_filename, json_filepath, content_key='abstract', meta_keys=[
                'article_title', 'journal',
                # 'keywords', 'mesh_headings', 'major_topics'
                ], logger=None, logging_level=logging.INFO
            ):
        self.logger = create_function_logger(__name__, parent_logger=logger, level=logging_level)
        dictionary_list = load_json(json_filename, json_filepath)[:5]
        raw_docs = []
        self.logger.info(f'***Instantiating `Index_Docs`***')
        for dictionary in dictionary_list:
            doc = Document(content=dictionary[content_key], meta={key: dictionary[key] for key in meta_keys})
            raw_docs.append(doc)
        self.raw_docs = raw_docs
        self.logger.info(f'Initialized Index_Docs with {len(raw_docs)} documents')
    
    def run_pipeline(self, indexing_pipeline, first_component_name='cleaner'):
        self.logger.info(f'***Running indexing pipeline***')
        indexing_pipeline.run({first_component_name: {"documents": self.raw_docs}})

if __name__ == "__main__":
    logger = create_function_logger(__name__, parent_logger=None, level=logging.INFO)
    filename = 'prod_pubmed_results_2024-04-07_112508.json'
    collection_name = 'prod_subset'
    filepath = '/home/silvhua/repositories/pubmed-search/data/'
    document_store = ChromaDocumentStore( # https://docs.haystack.deepset.ai/reference/integrations-chroma#chromadocumentstore
        collection_name=collection_name, 
        persist_path='../data/processed/'
        )
    
    metadata_fields = ['article_title', 'journal']
    
    indexer = Index_Docs(
        filename, filepath, meta_keys=metadata_fields
        )
    indexing_pipeline = create_indexing_pipeline(
        document_store, metadata_fields=metadata_fields
        )
    indexer.run_pipeline(indexing_pipeline, first_component_name='cleaner')
    logger.info(f'Finished indexing pipeline')

2024-04-09 16:03:33,324 - __main__ - INFO:
***Instantiating `Index_Docs`***

2024-04-09 16:03:33,327 - __main__ - INFO:
Initialized Index_Docs with 5 documents

2024-04-09 16:03:33,331 - __main__ - INFO:
***Running indexing pipeline***

/home/silvhua/.pubmed/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/silvhua/.pubmed/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

2024-04-09 16:04:16,932 - __main__ - INFO:
Finished indexing pipeline



In [5]:
vars(document_store)

{'_collection_name': 'prod_subset',
 '_embedding_function': 'default',
 '_embedding_function_params': {},
 '_persist_path': '../data/processed/',
 '_chroma_client': <chromadb.api.client.Client at 0x7f8ffa9cf4f0>,
 '_collection': Collection(name=prod_subset)}

In [7]:
vars(document_store._chroma_client)

{'_identifier': '../data/processed/',
 'tenant': 'default_tenant',
 'database': 'default_database',
 '_admin_client': <chromadb.api.client.AdminClient at 0x7f8ffa9cf130>,
 '_server': <chromadb.api.segment.SegmentAPI at 0x7f8ffa9cf1f0>}

## retrieval pipeline

In [9]:
from haystack_integrations.components.retrievers.chroma import ChromaEmbeddingRetriever
from haystack.components.embedders import SentenceTransformersTextEmbedder

# https://docs.haystack.deepset.ai/docs/chromaembeddingretriever
retriever = ChromaEmbeddingRetriever(document_store=document_store, top_k=3)

retrieval_pipeline = Pipeline()
retrieval_pipeline.add_component("text_embedder", SentenceTransformersTextEmbedder(
    # model="thenlper/gte-large"
    ))
retrieval_pipeline.add_component("retriever_with_embeddings", retriever)

retrieval_pipeline.connect("text_embedder", "retriever_with_embeddings")

In [11]:
vars(retriever.document_store)

{'_collection_name': 'prod_subset',
 '_embedding_function': 'default',
 '_embedding_function_params': {},
 '_persist_path': '../data/processed/',
 '_chroma_client': <chromadb.api.client.Client at 0x7f8ffa9cf4f0>,
 '_collection': Collection(name=prod_subset)}

### Chroma Collections
https://docs.trychroma.com/api-reference#methods-related-to-collections

In [14]:
retriever.document_store._collection.count()

37

In [17]:
retriever.document_store._collection.schema_json()

'{"title": "Collection", "type": "object", "properties": {"name": {"title": "Name", "type": "string"}, "id": {"title": "Id", "type": "string", "format": "uuid"}, "metadata": {"title": "Metadata", "type": "object"}, "tenant": {"title": "Tenant", "type": "string"}, "database": {"title": "Database", "type": "string"}}, "required": ["name", "id"]}'

In [18]:
retriever.document_store._collection.get()

{'ids': ['a9e557662d691b0f57b0c98af171d81e0e0fb6aec823174d980a61a1fb2c624d',
  'eae58438bfc0db3c361a2ece7cc420afb6247aee2f3994455ec3220116047cf8',
  '714dcebf7b99e5461c1d367d8538d730c05a31b02cdd5eb1c2a9b0c096191940',
  '8f4af2cc72f92907c1d49574246e588f72b818c316a435aeb536e152d8d18eee',
  '1c637f819e8bfb8c8c8af91c01b67de5a533381e06f16d7c11a63ae85bfe3171',
  'c1dddb2fcd6b25fbe0874713f9d4f7a71da2faaf48dfe97fe2965879097f29a4',
  '279b64718528f9d5d220a96eb68cc2c56d9a9917d2b1cc43ecc633eb84cd9e25',
  '3c76f724bbe4094739c212ff4e4ea71931ad255553d980399636c62e91061091',
  '87bf10a918f3d21420f4f399f66fe21a3f935c33ce438608d7cb8df7bc319a52',
  'df7127e8f9106edf69a63f61c6752720e2c8487e84d02a3a1214ffa15f7137bc',
  '8ef027115421c5eacc754d09ee4a352003027c97543b82c2db3b9f16a94b84de',
  '47b00388908a01042a4b0e956536cd3f579f5421e3d9ca388aa0d7a2e48c77f3',
  '5d8e91a1f379760a9106d6bcf1778161e2f54709fd8d56024c5d802d1141b810',
  'dc69d2178b6590633a5f659cd9c767e3a5bf6d70f0f140e519c849428130ebb9',
  '8fae425c3f

In [19]:
retriever.document_store._collection.get().keys()

dict_keys(['ids', 'embeddings', 'metadatas', 'documents', 'uris', 'data'])

In [21]:
doc_metadata_list = retriever.document_store._collection.get()['metadatas']
len(doc_metadata_list)

37

In [20]:
retriever.document_store._collection.get()['documents']

['OBJECTIVE: Chronic kidney disease (CKD) is a condition influenced by both genetic and environmental factors and has been a focus of extensive research. Utilizing Mendelian randomization, researchers have begun to untangle the complex causal relationships underlying CKD.',
 ' This review delves into the advances and challenges in the application of MR in the field of nephrology, shifting from a mere summary of its principles and limitations to a more nuanced exploration of its contributions to our understanding of CKD.<br>METHODS: Key findings from recent studies have been pivotal in reshaping our comprehension of CKD.',
 ' Notably, evidence indicates that elevated testosterone levels may impair renal function, while higher sex hormone-binding globulin (SHBG) levels appear to be protective, predominantly in men. Surprisingly, variations in plasma glucose and glycated hemoglobin levels seem unaffected by genetically induced changes in the estimated glomerular filtration rate (eGFR), su

### Open existing document store

In [22]:
document_store2 = ChromaDocumentStore( # https://docs.haystack.deepset.ai/reference/integrations-chroma#chromadocumentstore
        collection_name='test_set', 
        persist_path='../data/processed/'
        )

In [23]:
documents2 = document_store2._collection.get()['documents']

In [25]:
document_store2._collection.count()

125

In [24]:
documents2

['PURPOSE: After hospitalization, older persons may face a decline in physical function and daily independence. In-hospital exercise interventions can mitigate this decline, and continued support from primary healthcare post-discharge may enhance sustainability.',
 " This study aimed to explore home rehabilitation therapists' experiences of supporting physical exercise after acute hospitalization, including exercise programs initiated during hospital stay. METHODS: This qualitative study was conducted alongside a randomized-controlled trial to investigate prerequisites for a transitional care intervention.",
 ' Twelve interviews were conducted with physiotherapists, occupational therapists, and managers across seven rehabilitation therapy services in Stockholm, Sweden. Data were analyzed using reflexive thematic analysis.',
 " RESULTS: The analysis generated the theme Striving for individualized support for physical exercise, although limited resources and a fragmented home care risk t

In [ ]:
retriever2 = ChromaEmbeddingRetriever(document_store=document_store2, top_k=3)

retrieval_pipeline2 = Pipeline()
retrieval_pipeline2.add_component("text_embedder", SentenceTransformersTextEmbedder(
    # model="thenlper/gte-large"
    ))
retrieval_pipeline2.add_component("retriever_with_embeddings", retriever2)

retrieval_pipeline2.connect("text_embedder", "retriever_with_embeddings")

# *End of Page*